# Olá! Fico muito feliz por sua presença.

A seguir, você verá uma **Análise RFV** de um negócio hipotético, desenvolvida com a linguagem de programação Python.


Para que você possa praticar, disponibilizei a base de dados para download.

**Aproveite o material e bons estudos!**

Autor: *Igor Girão Saraiva* www.linkedin.com/in/igor-girão

In [404]:
#Importando bibliotecas
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 1º Passo – Preparação dos Dados

Carregar a base de dados, identificar possíveis **erros e inconsistências** e, em seguida, realizar o tratamento necessário para garantir uma análise precisa.

In [405]:
#Importando base de dados
df_base = pd.read_excel('analise_rfv.xlsx')
df_base.head()

,ID Cliente,Data Primeira Compra,Valor Primeira Compra,Mês 1,Mês 2,Mês 3,Mês 4,Mês 5,Mês 6,Mês 7,Mês 8,Mês 9,Mês 10,Mês 11,Mês 12
0,1,2022-11-01,470.27,0.00,0.00,434.74,0.00,0.00,118.99,194.73,426.34,0.00,0.00,0.00,0.00
1,2,2022-09-30,155.15,72.62,0.00,80.94,0.00,71.12,0.00,177.96,0.00,65.88,113.66,50.75,0.00
2,3,2022-02-20,164.90,91.24,47.85,152.47,134.11,61.99,0.00,0.00,0.00,0.00,135.65,0.00,49.85
3,4,2022-01-18,60.67,0.00,47.80,69.20,51.55,0.00,9.46,65.52,70.41,50.65,0.00,0.00,46.51
4,5,2022-05-22,396.21,317.40,122.35,131.84,0.00,0.00,122.78,84.60,0.00,95.41,0.00,51.55,205.04


In [406]:
#Corrigindo tabela
df_base = df_base.drop(['Valor Primeira Compra','Data Primeira Compra'], axis=1)
df_base.head()

,ID Cliente,Mês 1,Mês 2,Mês 3,Mês 4,Mês 5,Mês 6,Mês 7,Mês 8,Mês 9,Mês 10,Mês 11,Mês 12
0,1,0.00,0.00,434.74,0.00,0.00,118.99,194.73,426.34,0.00,0.00,0.00,0.00
1,2,72.62,0.00,80.94,0.00,71.12,0.00,177.96,0.00,65.88,113.66,50.75,0.00
2,3,91.24,47.85,152.47,134.11,61.99,0.00,0.00,0.00,0.00,135.65,0.00,49.85
3,4,0.00,47.80,69.20,51.55,0.00,9.46,65.52,70.41,50.65,0.00,0.00,46.51
4,5,317.40,122.35,131.84,0.00,0.00,122.78,84.60,0.00,95.41,0.00,51.55,205.04


In [407]:
#Desimpilhando colunas
df = df_base.melt(id_vars=['ID Cliente'], var_name='Mês', value_name='Valor Compra')

# Convertendo 'Mês' para uma data
df['Mês'] = df['Mês'].str.replace('Mês ', '').astype(int)
df['Data Compra'] = pd.to_datetime(df['Mês'].astype(str) + '-01-2024', format='%m-%d-%Y')


# Ordenando os dados
df = df.sort_values(by=['ID Cliente', 'Mês'])

# Exibir resultado (SEMPRE GOSTO DE CONFERIR MINHAS ETAPAS)
df

,ID Cliente,Mês,Valor Compra,Data Compra
0,1,1,0.00,2024-01-01
1000,1,2,0.00,2024-02-01
2000,1,3,434.74,2024-03-01
3000,1,4,0.00,2024-04-01
4000,1,5,0.00,2024-05-01
...,...,...,...,...
7999,1000,8,155.74,2024-08-01
8999,1000,9,42.25,2024-09-01
9999,1000,10,11.82,2024-10-01
10999,1000,11,0.00,2024-11-01


In [408]:
#Removendo linhas com compras nulas
df = df.query('`Valor Compra` != 0')
df.isnull().sum()#Conferindo com isnull

,0
ID Cliente,0
Mês,0
Valor Compra,0
Data Compra,0


In [409]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5942 entries, 2000 to 11999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID Cliente    5942 non-null   int64         
 1   Mês           5942 non-null   int64         
 2   Valor Compra  5942 non-null   float64       
 3   Data Compra   5942 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 232.1 KB


# 2º Passo – Transformando Dados em Informações

Com a tabela já processada, podemos agora iniciar nossa análise, categorizando os clientes em grupos com base nos critérios de **Recência (R), Frequência (F) e Valor (V)**.

In [410]:
#A partir de agora trabalharemos com a nossa tabela df agrupada pois ela permite parametrizar nossos clientes
df_agrupado = df.groupby('ID Cliente').agg(
    Valor_Médio_Gasto=('Valor Compra', 'mean'),
    Data_Ultima_Compra=('Data Compra', 'max'),
    Qtd_Compras=('Valor Compra', 'count')
).reset_index()

#Arredondando duas casas decimais para Valor_Médio_Gasto
df_agrupado['Valor_Médio_Gasto'] = df_agrupado['Valor_Médio_Gasto'].round(2)

df_agrupado.head()

,ID Cliente,Valor_Médio_Gasto,Data_Ultima_Compra,Qtd_Compras
0,1,293.70,2024-08-01,4
1,2,90.42,2024-11-01,7
2,3,96.17,2024-12-01,7
3,4,51.39,2024-12-01,8
4,5,141.37,2024-12-01,8


**RECÊNCIA:** Refere-se ao tempo decorrido desde a última compra do cliente em sua loja.

In [411]:
#Subtraindo a data de hoje (30/12/2025) com a data da última compra para encontrar um valor em dias
data_ref = pd.to_datetime('2024-12-30')

df_agrupado['Dias Desde Compra'] = (data_ref - df_agrupado['Data_Ultima_Compra']).dt.days
df_agrupado = df_agrupado.sort_values(by=['Dias Desde Compra'])
df_agrupado.head() #Sempre conferindo

,ID Cliente,Valor_Médio_Gasto,Data_Ultima_Compra,Qtd_Compras,Dias Desde Compra
499,500,68.04,2024-12-01,7,29
542,543,181.88,2024-12-01,7,29
544,545,117.76,2024-12-01,8,29
548,549,70.52,2024-12-01,6,29
549,550,204.51,2024-12-01,6,29


In [412]:
#Categorizando os clientes de acordo com a recência
Categoria_recencia = [1,2,3,4,5]
df_agrupado['RECENCIA'] = 0 #Criando coluna Recencia
#Atribuindo valores de RECÊNCIA
for a in df_agrupado.index: #utilize o df_agrupado.index uma vez que já excluímos os valores nulos, assim, seu loop não passará por campos vazios.
    if df_agrupado['Dias Desde Compra'][a] <= 30:
        df_agrupado['RECENCIA'][a] = Categoria_recencia[4]
    elif 30 < df_agrupado['Dias Desde Compra'][a] <= 60:
        df_agrupado['RECENCIA'][a] = Categoria_recencia[3]
    elif 60 < df_agrupado['Dias Desde Compra'][a] <= 120:
        df_agrupado['RECENCIA'][a] = Categoria_recencia[2]
    elif 120 < df_agrupado['Dias Desde Compra'][a] <= 180:
        df_agrupado['RECENCIA'][a] = Categoria_recencia[1]
    elif 180 < df_agrupado['Dias Desde Compra'][a] <= 360:
        df_agrupado['RECENCIA'][a] = Categoria_recencia[0]

df_agrupado.head()#Sugiro sempre printar df.tail() para verificar últimos valores.

,ID Cliente,Valor_Médio_Gasto,Data_Ultima_Compra,Qtd_Compras,Dias Desde Compra,RECENCIA
499,500,68.04,2024-12-01,7,29,5
542,543,181.88,2024-12-01,7,29,5
544,545,117.76,2024-12-01,8,29,5
548,549,70.52,2024-12-01,6,29,5
549,550,204.51,2024-12-01,6,29,5


**FREQUÊNCIA:** Refere-se ao número de vezes que o cliente realizou compras em seu estabelecimento.

In [413]:
#Categorizando os clientes de acordo com a frequência
Categoria_frequência = [1,2,3,4,5]
df_agrupado['FREQUENCIA'] = 0 #Criando coluna frequência
#Atribuindo valores
for a in df_agrupado.index:
    if df_agrupado['Qtd_Compras'][a] >= 12:
        df_agrupado['FREQUENCIA'][a] = Categoria_frequência[4]
    elif 12 > df_agrupado['Qtd_Compras'][a] >= 11:
        df_agrupado['FREQUENCIA'][a] = Categoria_frequência[3]
    elif 11 > df_agrupado['Qtd_Compras'][a] >= 8:
        df_agrupado['FREQUENCIA'][a] = Categoria_frequência[2]
    elif 8 > df_agrupado['Qtd_Compras'][a] >= 5:
        df_agrupado['FREQUENCIA'][a] = Categoria_frequência[1]
    elif 5 > df_agrupado['Qtd_Compras'][a] >= 1:
        df_agrupado['FREQUENCIA'][a] = Categoria_frequência[0]

df_agrupado.head() #Sugiro sempre printar df.tail() para verificar últimos valores.

,ID Cliente,Valor_Médio_Gasto,Data_Ultima_Compra,Qtd_Compras,Dias Desde Compra,RECENCIA,FREQUENCIA
499,500,68.04,2024-12-01,7,29,5,2
542,543,181.88,2024-12-01,7,29,5,2
544,545,117.76,2024-12-01,8,29,5,3
548,549,70.52,2024-12-01,6,29,5,2
549,550,204.51,2024-12-01,6,29,5,2


**VALOR:** A categorização de valor está relacionada ao poder aquisitivo do cliente e à forma como ele pode ser segmentado com base nesse critério.

In [414]:
#Categorizando os clientes de acordo com o poder aquisitivo
#Calculando os percentis

Porcent_Valor = [0.1 , 0.2 , 0.4 , 0.7 , 0.9]
Parametros_Valor = []

for i in range(5):
   a = round(df_agrupado['Valor_Médio_Gasto'].quantile(1-Porcent_Valor[i]), 2)
   Parametros_Valor.append(a)

Parametros_Valor

[286.93, 236.14, 181.41, 106.77, 54.84]

In [415]:
#Categorizando os clientes de acordo com o valor
#Valores de referencia [286.93, 236.13, 181.41, 106.77, 54.84]
Categoria_Valor = [1,2,3,4,5]
df_agrupado['VALOR'] = 0 #Criando coluna frequência
#Atribuindo valores
for a in df_agrupado.index:
    if df_agrupado['Valor_Médio_Gasto'][a] >= 286.93:
        df_agrupado['VALOR'][a] = Categoria_Valor[4]
    elif 286.93 > df_agrupado['Valor_Médio_Gasto'][a] >= 236.13:
        df_agrupado['VALOR'][a] = Categoria_Valor[3]
    elif 236.1311 > df_agrupado['Valor_Médio_Gasto'][a] >= 181.41:
        df_agrupado['VALOR'][a] = Categoria_Valor[2]
    elif 181.44 > df_agrupado['Valor_Médio_Gasto'][a] >= 106.77:
        df_agrupado['VALOR'][a] = Categoria_Valor[1]
    elif 106.77 > df_agrupado['Valor_Médio_Gasto'][a] >= 54.84:
        df_agrupado['VALOR'][a] = Categoria_Valor[0]

df_agrupado.head() #Sugiro sempre printar df.tail() para verificar últimos valores.

,ID Cliente,Valor_Médio_Gasto,Data_Ultima_Compra,Qtd_Compras,Dias Desde Compra,RECENCIA,FREQUENCIA,VALOR
499,500,68.04,2024-12-01,7,29,5,2,1
542,543,181.88,2024-12-01,7,29,5,2,3
544,545,117.76,2024-12-01,8,29,5,3,2
548,549,70.52,2024-12-01,6,29,5,2,1
549,550,204.51,2024-12-01,6,29,5,2,3


In [416]:
#Atribuindo "SCORE RFV" aos meus clientes
df_agrupado['SCORE RFV'] = df_agrupado['RECENCIA'] + df_agrupado['FREQUENCIA'] + df_agrupado['VALOR']
df_agrupado.head()

,ID Cliente,Valor_Médio_Gasto,Data_Ultima_Compra,Qtd_Compras,Dias Desde Compra,RECENCIA,FREQUENCIA,VALOR,SCORE RFV
499,500,68.04,2024-12-01,7,29,5,2,1,8
542,543,181.88,2024-12-01,7,29,5,2,3,10
544,545,117.76,2024-12-01,8,29,5,3,2,10
548,549,70.52,2024-12-01,6,29,5,2,1,8
549,550,204.51,2024-12-01,6,29,5,2,3,10


# 3º Passo – Síntese e Relatório

Com todas as informações tratadas e geradas, podemos agora resumir os dados e elaborar nosso relatório.

A partir deste ponto, você tem total liberdade para explorar os dados da maneira que preferir. Aqui, apresentarei apenas os insights mais essenciais.

**Lembre-se: menos é mais!**

In [417]:
#Importando bibliotecas para visualização de dados
!pip install plotly==5.15.0
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [418]:
#Mostrando Valor Total das Vendas
valor_total = df['Valor Compra'].sum()

#Criando cartão indicador Total de Vendas
fig = go.Figure(go.Indicator(
    mode="number+delta",
    value=valor_total,
    title={"text": "Valor Total de Vendas"},
    domain={'x': [0, 1], 'y': [0, 1]}
))

fig.update_layout( height = 200, width=300,
    paper_bgcolor="lightgray",
    font=dict(size=15,color="darkblue", family="Arial")
)

fig.show()

In [419]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Recência", "Frequência", "Valor"))

# Gráfico de barras para Recência
fig.add_trace(
    go.Bar(x=df_agrupado['RECENCIA'].unique(),
           y=df_agrupado['RECENCIA'].value_counts().sort_index(),
           text=df_agrupado['RECENCIA'].value_counts().sort_index(),
           textposition='inside',  # Exibir resultado na extremidade interna
           marker_color='steelblue'),  # Cor azul aço
    row=1, col=1
)

# Gráfico de barras para Frequência
fig.add_trace(
    go.Bar(x=df_agrupado['FREQUENCIA'].unique(),
           y=df_agrupado['FREQUENCIA'].value_counts().sort_index(),
           text=df_agrupado['FREQUENCIA'].value_counts().sort_index(),
           textposition='inside',  # Exibir resultado na extremidade interna
           marker_color='darkorange'),  # Cor laranja escuro
    row=1, col=2
)

# Gráfico de barras para Valor
fig.add_trace(
    go.Bar(x=df_agrupado['VALOR'].unique(),
           y=df_agrupado['VALOR'].value_counts().sort_index(),
           text=df_agrupado['VALOR'].value_counts().sort_index(),
           textposition='inside',  # Exibir resultado na extremidade interna
           marker_color='forestgreen'),  # Cor verde floresta
    row=1, col=3
)

fig.update_layout(height=400, width=1200, showlegend=False, # remove a legenda
                  title_text="Análise RFV: Distribuição de Clientes")

fig.show()

In [420]:
fig = px.scatter(df_agrupado, x="RECENCIA", y="FREQUENCIA",
                 title="Relação entre Recência e Frequência",
                 labels={'RECENCIA': 'Recência', 'FREQUENCIA': 'Frequência'},
                 color="SCORE RFV",  # Colorir os pontos pelo Score RFV
                 color_continuous_scale='Viridis', # Escala de cores
                 hover_data=['ID Cliente'])  # Mostrar ID do cliente ao passar o mouse


fig.update_traces(marker_size=10)
fig.update_layout(xaxis_title='Recência', yaxis_title='Frequência')

fig.show()

In [421]:
fig = px.scatter_3d(df_agrupado, x='RECENCIA', y='FREQUENCIA', z='VALOR',
                    color='Valor_Médio_Gasto', size='Qtd_Compras',
                    color_continuous_scale='Viridis',
                    labels={'RECENCIA': 'Recência', 'FREQUENCIA': 'Frequência', 'VALOR': 'Valor',
                            'Valor_Médio_Gasto': 'Valor Médio Gasto', 'Qtd_Compras': 'Quantidade de Compras'},
                    title='Gráfico RFV 3D com Gradiente de Cor')

fig.update_layout(height=800, width=1200)
fig.show()